In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
startbucks3=pd.read_csv("startbucks_final_Phoebe.csv")

In [3]:
startbucks3['city']=startbucks3['city'].str.lower()

In [4]:
startbucks3.columns

Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule',
       'slug', 'latitude', 'longitude', 'streetAddressLine1',
       'streetAddressLine2', 'streetAddressLine3', 'city',
       'countrySubdivisionCode', 'postalCode', 'currentTimeOffset',
       'windowsTimeZoneId', 'olsonTimeZoneId', 'GMT', 'GMT_city', 'Mon', 'Tue',
       'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
      dtype='object')

In [5]:
startbucks3=startbucks3.drop(['slug', 'currentTimeOffset', 'schedule', 'olsonTimeZoneId'],axis=1)

In [6]:
#startbucks3.rename(columns={'GMT_city': 'GMT_region'}, inplace=True)

In [7]:
day_list=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [8]:
for x in day_list:
    time = startbucks3[x].str.split(' to ', expand=True)
    openname = f'{x}_open'
    closename = f'{x}_close'
    startbucks3[openname]=time[0]
    startbucks3[closename]=time[1]
    startbucks3=startbucks3.drop([x],axis=1)

In [18]:
startbucks3.iloc[:,15:]

,Tue_open,Tue_close,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
0,08:00,15:00,08:00,15:00,08:00,15:00,08:00,15:00,08:00,15:00,08:00,15:00
1,05:30,22:00,05:30,22:00,05:30,22:00,05:30,22:00,06:00,21:00,06:00,21:00
2,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00
3,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00
4,05:30,21:00,05:30,21:00,05:30,21:00,05:30,21:00,06:00,21:00,06:00,20:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3789,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,10:30,20:00
3790,10:30,18:00,10:30,18:00,10:30,18:00,08:00,19:30,08:00,19:30,08:00,19:30
3791,10:00,19:00,10:00,19:00,10:00,19:00,10:00,19:00,10:00,19:00,Closed,Closed
3792,09:30,18:00,09:30,18:00,09:30,18:00,09:30,20:00,09:30,20:00,Closed,Closed


In [10]:
for n in day_list:
    openname = f'{n}_open'
    closename = f'{n}_close'
    startbucks3.loc[startbucks3[openname]=='Closed', closename] = 'Closed'
    startbucks3['time']=startbucks3[openname].str.extract(r'(\d{1,2}:\d{2}\s.M)')
    startbucks3.loc[startbucks3[openname].str.contains('M', regex=True)==True, openname] = pd.to_datetime(startbucks3['time']).dt.strftime('%H:%M')
    startbucks3['time']=startbucks3[closename].str.extract(r'(\d{1,2}:\d{2}\s.M)')
    startbucks3.loc[startbucks3[closename].str.contains('M', regex=True)==True, closename] = pd.to_datetime(startbucks3['time']).dt.strftime('%H:%M')


In [11]:
startbucks3=startbucks3.drop(['time'],axis=1)

In [12]:
startbucks3.columns

Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude',
       'longitude', 'streetAddressLine1', 'streetAddressLine2',
       'streetAddressLine3', 'city', 'countrySubdivisionCode', 'postalCode',
       'windowsTimeZoneId', 'GMT', 'GMT_region', 'Mon_open', 'Mon_close',
       'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close', 'Thu_open',
       'Thu_close', 'Fri_open', 'Fri_close', 'Sat_open', 'Sat_close',
       'Sun_open', 'Sun_close'],
      dtype='object')

In [13]:
address=startbucks3.columns.get_loc('streetAddressLine1')
col_name=startbucks3.columns.tolist()
col_name.insert(address,'address')
startbucks3=startbucks3.reindex(columns=col_name)

In [14]:
startbucks3.loc[startbucks3['streetAddressLine2']!='Null', 'address'] = startbucks3['streetAddressLine1'].str.cat(startbucks3['streetAddressLine2'],sep='|')
startbucks3.loc[startbucks3['streetAddressLine2']=='Null', 'address'] = startbucks3['streetAddressLine1']
startbucks3.loc[startbucks3['streetAddressLine3']!='Null', 'address'] = startbucks3['address'].str.cat(startbucks3['streetAddressLine3'],sep='|')

In [15]:
startbucks3=startbucks3.drop(['streetAddressLine1', 'streetAddressLine2','streetAddressLine3'],axis=1)


In [16]:
startbucks3 = startbucks3.fillna('Null')

In [19]:
startbucks3.to_csv('startbucks_final_Phoebe2.csv', index=False)